## Align only PE reads using `HISAT2`

In [ ]:
REF_DIR="~/data1/ref/Caenorhabditis_elegans"
BASE_DIR="../data"
INDEX="$REF_DIR/wbcel235/genome"

hisat_exe="$HOME/bin/hisat2-2.1.0/hisat2"

for sample in $(ls $BASE_DIR | grep ^ig)
do
    sample_dir="$BASE_DIR/$sample/trim"
    result_dir="../results/$sample"
    
    if [ ! -d "$result_dir" ]; then
        echo "Creating $result_dir ..."
        mkdir -p $result_dir
    fi
    
    echo "Unzipping reads for $sample ..."
    read1=$sample_dir/read1.fifo
    read2=$sample_dir/read2.fifo
    mkfifo $read1
    mkfifo $read2
    
    zcat $sample_dir/R1_pe.fastq.gz > $read1 &
    zcat $sample_dir/R2_pe.fastq.gz > $read2 &
    
    $hisat_exe -p 30 --dta \
        -x $INDEX -1 $read1 -2 $read2 \
        | samtools view -bhS - > "$result_dir/$sample.pe.bam"
    
    rm $sample_dir/*.fifo 
    
done

In [ ]:
for sample in $(ls $BASE_DIR | grep ^ig)
do
    result_dir="../results/$sample"
    samtools sort -@ 30 -o "$result_dir/$sample.pe.sorted.bam" "$result_dir/$sample.pe.bam"
    samtools index -@ 30 "$result_dir/$sample.pe.sorted.bam"
done

## Count reads in genes using `HTSeq-count`

In [ ]:
genes="~/data1/ref/Caenorhabditis_elegans/Ensembl/WBcel235/Annotation/Genes/genes.gtf"
BASE_DIR="../results"

counter=$(which htseq-count)

for sample in $(ls $BASE_DIR | grep ^ig)
do
    echo "Running HTSeq-count for $sample ..."
    echo "$BASE_DIR/$sample/$sample.pe.sorted.bam"
    
    $counter -f bam -q "$BASE_DIR/$sample/$sample.pe.sorted.bam" $genes \
        > "$BASE_DIR/$sample/$sample.htseq.counts" &
    
done